In [8]:
# Tro to batch the data using xbatcher instead:
import xbatcher as xb
import xarray as xr

import sys
sys.path.append("../")

from src.processing import windsat_datacube

In [9]:
folder_path = "../data/raw/Daily_Windsat/"
ds = windsat_datacube(folder_path)
ds

<xarray.Dataset> Size: 3GB
Dimensions:         (day_number: 21, longitude_grid: 1440, latitude_grid: 720,
                     swath_sector: 2, frequency_band: 2, polarization: 2)
Coordinates:
    lat             (latitude_grid) float32 3kB dask.array<chunksize=(720,), meta=np.ndarray>
    lon             (longitude_grid) float32 6kB dask.array<chunksize=(1440,), meta=np.ndarray>
  * day_number      (day_number) int32 84B 1 2 3 4 5 6 7 ... 16 17 18 19 20 21
Dimensions without coordinates: longitude_grid, latitude_grid, swath_sector,
                                frequency_band, polarization
Data variables: (12/13)
    longitude       (day_number, longitude_grid) float32 121kB dask.array<chunksize=(1, 1440), meta=np.ndarray>
    latitude        (day_number, latitude_grid) float32 60kB dask.array<chunksize=(1, 720), meta=np.ndarray>
    node            (day_number, swath_sector) int32 168B dask.array<chunksize=(1, 2), meta=np.ndarray>
    frequency_vpol  (day_number, frequency_band) float32 168B dask.array<chunksize=(1, 2), meta=np.ndarray>
    frequency_hpol  (day_number, frequency_band) float32 168B dask.array<chunksize=(1, 2), meta=np.ndarray>
    eia_nominal     (day_number, frequency_band) float32 168B dask.array<chunksize=(1, 2), meta=np.ndarray>
    ...              ...
    eaa             (day_number, frequency_band, latitude_grid, longitude_grid, swath_sector) float32 348MB dask.array<chunksize=(1, 1, 360, 720, 1), meta=np.ndarray>
    eia             (day_number, frequency_band, latitude_grid, longitude_grid, swath_sector) float32 348MB dask.array<chunksize=(1, 1, 360, 720, 1), meta=np.ndarray>
    tbtoa           (day_number, polarization, frequency_band, latitude_grid, longitude_grid, swath_sector) float32 697MB dask.array<chunksize=(1, 1, 1, 240, 480, 1), meta=np.ndarray>
    quality_flag    (day_number, latitude_grid, longitude_grid, swath_sector) int32 174MB dask.array<chunksize=(1, 360, 720, 1), meta=np.ndarray>
    surtep_ERA5     (day_number, latitude_grid, longitude_grid, swath_sector) float32 174MB dask.array<chunksize=(1, 360, 720, 1), meta=np.ndarray>
    airtep_ERA5     (day_number, latitude_grid, longitude_grid, swath_sector) float32 174MB dask.array<chunksize=(1, 360, 720, 1), meta=np.ndarray>
Attributes: (12/72)
    Conventions:                            CF-1.7
    title:                                  RSS WindSat TOA Brightness Temper...
    version:                                V01.0
    summary:                                The dataset contains RSS WindSat ...
    references:                              [1] T. Meissner et al., Remote S...
    acknowledgement:                        Funded under NASA Grant 80NSSC21K...
    ...                                     ...
    Source_of_ancillary_IMERG_rain_rate:    Huffman, G. et al.,  2019. NASA G...
    Source_of_ancillary_CCMP_wind:          Mears, C. et al., 2023.Remote Sen...
    Source_of_ancillary_ERA5:               ECMWF Reanalysis v5 (ERA5). https...
    Source_of_RSS_WindSat_AS_ECV:           https://www.remss.com/missions/wi...
    Source_of_ancillary_HYCOM_SSS:          Hybrid Coordinate Ocean Model, Gl...
    Source_of_WindSat_L1C_TB:               T. Meissner et al., Remote Sensin...

In [108]:
# Select where there is tbtoa data.
ascds = ds.where(ds.notnull(), drop=True).sel(swath_sector = 0)
# ascds.tbtoa.sel(day_number=1, polarization = 0, frequency_band = 0).plot()

KeyError: 'Indexing with a boolean dask array is not allowed. This will result in a dask array of unknown shape. Such arrays are unsupported by Xarray.Please compute the indexer first using .compute()'

In [90]:
asc_bgen = xb.BatchGenerator(
    ds=ascds,
    input_dims ={"latitude_grid":1,"longitude_grid":1, "day_number":21},
    )


In [92]:
print(len(asc_bgen))
asc_bgen[0].sel(sample=0)

training_example = asc_bgen[0][["tbtoa","time","longitude","latitude"]]

1036800


In [109]:
mask = ds.notnull()
mask = mask.compute()
fds = ds.where(mask, drop=True)
fds

<xarray.Dataset> Size: 3GB
Dimensions:         (day_number: 21, longitude_grid: 1440, latitude_grid: 720,
                     swath_sector: 2, frequency_band: 2, polarization: 2)
Coordinates:
    lat             (latitude_grid) float32 3kB -89.88 -89.62 ... 89.62 89.88
    lon             (longitude_grid) float32 6kB 180.1 180.4 ... 179.6 179.9
  * day_number      (day_number) int32 84B 1 2 3 4 5 6 7 ... 16 17 18 19 20 21
Dimensions without coordinates: longitude_grid, latitude_grid, swath_sector,
                                frequency_band, polarization
Data variables: (12/13)
    longitude       (day_number, longitude_grid) float32 121kB dask.array<chunksize=(1, 1440), meta=np.ndarray>
    latitude        (day_number, latitude_grid) float32 60kB dask.array<chunksize=(1, 720), meta=np.ndarray>
    node            (day_number, swath_sector) float64 336B dask.array<chunksize=(1, 2), meta=np.ndarray>
    frequency_vpol  (day_number, frequency_band) float32 168B 18.68 ... 37.02
    frequency_hpol  (day_number, frequency_band) float32 168B 18.73 ... 37.02
    eia_nominal     (day_number, frequency_band) float32 168B 55.57 ... 53.2
    ...              ...
    eaa             (day_number, frequency_band, latitude_grid, longitude_grid, swath_sector) float32 348MB dask.array<chunksize=(1, 1, 360, 720, 1), meta=np.ndarray>
    eia             (day_number, frequency_band, latitude_grid, longitude_grid, swath_sector) float32 348MB dask.array<chunksize=(1, 1, 360, 720, 1), meta=np.ndarray>
    tbtoa           (day_number, polarization, frequency_band, latitude_grid, longitude_grid, swath_sector) float32 697MB dask.array<chunksize=(1, 1, 1, 240, 480, 1), meta=np.ndarray>
    quality_flag    (day_number, latitude_grid, longitude_grid, swath_sector) float64 348MB dask.array<chunksize=(1, 360, 720, 1), meta=np.ndarray>
    surtep_ERA5     (day_number, latitude_grid, longitude_grid, swath_sector) float32 174MB dask.array<chunksize=(1, 360, 720, 1), meta=np.ndarray>
    airtep_ERA5     (day_number, latitude_grid, longitude_grid, swath_sector) float32 174MB dask.array<chunksize=(1, 360, 720, 1), meta=np.ndarray>
Attributes: (12/72)
    Conventions:                            CF-1.7
    title:                                  RSS WindSat TOA Brightness Temper...
    version:                                V01.0
    summary:                                The dataset contains RSS WindSat ...
    references:                              [1] T. Meissner et al., Remote S...
    acknowledgement:                        Funded under NASA Grant 80NSSC21K...
    ...                                     ...
    Source_of_ancillary_IMERG_rain_rate:    Huffman, G. et al.,  2019. NASA G...
    Source_of_ancillary_CCMP_wind:          Mears, C. et al., 2023.Remote Sen...
    Source_of_ancillary_ERA5:               ECMWF Reanalysis v5 (ERA5). https...
    Source_of_RSS_WindSat_AS_ECV:           https://www.remss.com/missions/wi...
    Source_of_ancillary_HYCOM_SSS:          Hybrid Coordinate Ocean Model, Gl...
    Source_of_WindSat_L1C_TB:               T. Meissner et al., Remote Sensin...

In [126]:
bgen = xb.BatchGenerator(
    ds=fds,
    input_dims ={"latitude_grid":100,"longitude_grid":100, "day_number":21},
    )


In [127]:
print(len(bgen))
bgen[0].to_dataframe()

98


longitude  \
day_number longitude_grid latitude_grid swath_sector frequency_band polarization              
1          0              0             0            0              0               180.125   
                                                                    1               180.125   
                                                     1              0               180.125   
                                                                    1               180.125   
                                        1            0              0               180.125   
...                                                                                     ...   
21         99             99            0            1              1               204.875   
                                        1            0              0               204.875   
                                                                    1               204.875   
                                                     1              0               204.875   
                                                                    1               204.875   

                                                                                  latitude  \
day_number longitude_grid latitude_grid swath_sector frequency_band polarization             
1          0              0             0            0              0              -89.875   
                                                                    1              -89.875   
                                                     1              0              -89.875   
                                                                    1              -89.875   
                                        1            0              0              -89.875   
...                                                                                    ...   
21         99             99            0            1              1              -65.125   
                                        1            0              0              -65.125   
                                                                    1              -65.125   
                                                     1              0              -65.125   
                                                                    1              -65.125   

                                                                                  node  \
day_number longitude_grid latitude_grid swath_sector frequency_band polarization         
1          0              0             0            0              0              1.0   
                                                                    1              1.0   
                                                     1              0              1.0   
                                                                    1              1.0   
                                        1            0              0              2.0   
...                                                                                ...   
21         99             99            0            1              1              1.0   
                                        1            0              0              2.0   
                                                                    1              2.0   
                                                     1              0              2.0   
                                                                    1              2.0   

                                                                                  frequency_vpol  \
day_number longitude_grid latitude_grid swath_sector frequency_band polarization                   
1          0              0             0            0              0                  18.681999   
                                                                    1                  18.681999   
                                                    

In [143]:
import pandas as pd
import numpy as np
from datetime import datetime
def transform_batch(batch: pd.DataFrame):

    # Select desired dvars:
    dvars = ["tbtoa","surtep_ERA5","lat","lon","time","tbtoa","surtep_ERA5"]
    batch = batch[dvars]

    # Remove missing data
    batch = batch.dropna()

    # We want the day_number value as an input
    batch.reset_index("day_number", inplace=True)

    # Transform the variables time, lon and day
    global_bias = (datetime(2017, 1, 1, 0, 0, 0) - datetime(2000, 1, 1, 0, 0)).total_seconds()
    batch["time"] += - global_bias - (batch["day_number"] - 1)* 24 * 60 * 60

    batch["time"] = batch["time"].apply(
        lambda x: np.sin(2 * np.pi * x / (24 * 60 * 60))
    )
    
    # Lon and lat transformations, to have a number between -1 and 1
    batch["lon"] = batch["lon"].apply(lambda x: np.sin(np.deg2rad(x)))
    batch["lat"] = batch["lat"].apply(lambda x: np.sin(np.deg2rad(x)))

    return batch

In [144]:
batch = bgen[0].to_dataframe()
batch = transform_batch(batch)
batch


day_number  \
longitude_grid latitude_grid swath_sector frequency_band polarization               
0              23            1            0              0                      1   
                                                         1                      1   
                                          1              0                      1   
                                                         1                      1   
               24            1            0              0                      1   
...                                                                           ...   
99             98            0            1              1                     21   
               99            0            0              0                     21   
                                                         1                     21   
                                          1              0                     21   
                                                         1                     21   

                                                                            tbtoa  \
longitude_grid latitude_grid swath_sector frequency_band polarization               
0              23            1            0              0             222.392502   
                                                         1             181.214996   
                                          1              0             227.417496   
                                                         1             200.302490   
               24            1            0              0             222.757507   
...                                                                           ...   
99             98            0            1              1             144.584991   
               99            0            0              0             186.017502   
                                                         1             107.767502   
                                          1              0             209.732498   
                                                         1             145.974991   

                                                                       surtep_ERA5  \
longitude_grid latitude_grid swath_sector frequency_band polarization                
0              23            1            0              0              260.173462   
                                                         1              260.173462   
                                          1              0              260.173462   
                                                         1              260.173462   
               24            1            0              0              261.280731   
...                                                                            ...   
99             98            0            1              1              273.224640   
               99            0            0              0              273.228607   
                                                         1              273.228607   
                                          1              0              273.228607   
                                                         1              273.228607   

                                                                            lat  \
longitude_grid latitude_grid swath_sector frequency_band polarization             
0              23            1            0              0            -0.994748   
                                                         1            -0.994748   
                                          1              0            -0.994748   
                                                         1            -0.994748   
               24            1            0              0            -0.994291   
...                                                                         ...   
99             98            0            1              1            -0.909

In [134]:
batch.reset_index("day_number")

day_number  \
longitude_grid latitude_grid swath_sector frequency_band polarization               
0              23            1            0              0                      1   
                                                         1                      1   
                                          1              0                      1   
                                                         1                      1   
               24            1            0              0                      1   
...                                                                           ...   
99             98            0            1              1                     21   
               99            0            0              0                     21   
                                                         1                     21   
                                          1              0                     21   
                                                         1                     21   

                                                                            tbtoa  \
longitude_grid latitude_grid swath_sector frequency_band polarization               
0              23            1            0              0             222.392502   
                                                         1             181.214996   
                                          1              0             227.417496   
                                                         1             200.302490   
               24            1            0              0             222.757507   
...                                                                           ...   
99             98            0            1              1             144.584991   
               99            0            0              0             186.017502   
                                                         1             107.767502   
                                          1              0             209.732498   
                                                         1             145.974991   

                                                                       surtep_ERA5  \
longitude_grid latitude_grid swath_sector frequency_band polarization                
0              23            1            0              0              260.173462   
                                                         1              260.173462   
                                          1              0              260.173462   
                                                         1              260.173462   
               24            1            0              0              261.280731   
...                                                                            ...   
99             98            0            1              1              273.224640   
               99            0            0              0              273.228607   
                                                         1              273.228607   
                                          1              0              273.228607   
                                                         1              273.228607   

                                                                          lat  \
longitude_grid latitude_grid swath_sector frequency_band polarization           
0              23            1            0              0            -84.125   
                                                         1            -84.125   
                                          1              0            -84.125   
                                                         1            -84.125   
               24            1            0              0            -83.875   
...                                                                       ...   
99             98            0            1              1            -65.375   
           